# Vanessa Williams
# Milestone 3

## Cleaning data from an API

In [103]:
!pip install requests pandas

In [104]:
import requests
import pandas as pd

In [21]:
url = "https://nfl-api-data.p.rapidapi.com/nfl-team-listing/v1/data"

headers = {
	"x-rapidapi-key": "546d4a34f1msh61af993ffc55cddp1aaecejsn67b84b3b171e",
	"x-rapidapi-host": "nfl-api-data.p.rapidapi.com"
}

teams_data = requests.get(url, headers=headers)

print(teams_data.json())

[{'team': {'id': '22', 'uid': 's:20~l:28~t:22', 'slug': 'arizona-cardinals', 'abbreviation': 'ARI', 'displayName': 'Arizona Cardinals', 'shortDisplayName': 'Cardinals', 'name': 'Cardinals', 'nickname': 'Cardinals', 'location': 'Arizona', 'color': 'a40227', 'alternateColor': 'ffffff', 'isActive': True, 'isAllStar': False, 'logos': [{'href': 'https://a.espncdn.com/i/teamlogos/nfl/500/ari.png', 'alt': '', 'rel': ['full', 'default'], 'width': 500, 'height': 500}, {'href': 'https://a.espncdn.com/i/teamlogos/nfl/500-dark/ari.png', 'alt': '', 'rel': ['full', 'dark'], 'width': 500, 'height': 500}, {'href': 'https://a.espncdn.com/i/teamlogos/nfl/500/scoreboard/ari.png', 'alt': '', 'rel': ['full', 'scoreboard'], 'width': 500, 'height': 500}, {'href': 'https://a.espncdn.com/i/teamlogos/nfl/500-dark/scoreboard/ari.png', 'alt': '', 'rel': ['full', 'scoreboard', 'dark'], 'width': 500, 'height': 500}], 'links': [{'language': 'en-US', 'rel': ['clubhouse', 'desktop', 'team'], 'href': 'https://www.espn.

In [93]:
# Step 4: Normalize the nested data to flatten it
df = pd.json_normalize(data)

# Step 5: Display the DataFrame in a readable format
df.head()  # Display the first few rows of the flattened DataFrame

,team.id,team.uid,team.slug,team.abbreviation,team.displayName,team.shortDisplayName,team.name,team.nickname,team.location,team.color,team.alternateColor,team.isActive,team.isAllStar,team.logos,team.links
0,22,s:20~l:28~t:22,arizona-cardinals,ARI,Arizona Cardinals,Cardinals,Cardinals,Cardinals,Arizona,a40227,ffffff,True,False,[{'href': 'https://a.espncdn.com/i/teamlogos/n...,"[{'language': 'en-US', 'rel': ['clubhouse', 'd..."
1,1,s:20~l:28~t:1,atlanta-falcons,ATL,Atlanta Falcons,Falcons,Falcons,Falcons,Atlanta,a71930,000000,True,False,[{'href': 'https://a.espncdn.com/i/teamlogos/n...,"[{'language': 'en-US', 'rel': ['clubhouse', 'd..."
2,33,s:20~l:28~t:33,baltimore-ravens,BAL,Baltimore Ravens,Ravens,Ravens,Ravens,Baltimore,29126f,000000,True,False,[{'href': 'https://a.espncdn.com/i/teamlogos/n...,"[{'language': 'en-US', 'rel': ['clubhouse', 'd..."
3,2,s:20~l:28~t:2,buffalo-bills,BUF,Buffalo Bills,Bills,Bills,Bills,Buffalo,00338d,d50a0a,True,False,[{'href': 'https://a.espncdn.com/i/teamlogos/n...,"[{'language': 'en-US', 'rel': ['clubhouse', 'd..."
4,29,s:20~l:28~t:29,carolina-panthers,CAR,Carolina Panthers,Panthers,Panthers,Panthers,Carolina,0085ca,000000,True,False,[{'href': 'https://a.espncdn.com/i/teamlogos/n...,"[{'language': 'en-US', 'rel': ['clubhouse', 'd..."


### Part 1: Replace Headers

In [97]:
# Rename columns (e.g., from 'team.name' to 'Team Name')
df.rename(columns={
    'team.name': 'Team Name',
    'team.nickname': 'Nickname',
    'team.location': 'Location'
}, inplace=True)

# Display the cleaned DataFrame
print(df.head())  # Display the first few rows of the renamed DataFrame

  team.id        team.uid          team.slug team.abbreviation  \
0      22  s:20~l:28~t:22  arizona-cardinals               ARI   
1       1   s:20~l:28~t:1    atlanta-falcons               ATL   
2      33  s:20~l:28~t:33   baltimore-ravens               BAL   
3       2   s:20~l:28~t:2      buffalo-bills               BUF   
4      29  s:20~l:28~t:29  carolina-panthers               CAR   

    team.displayName team.shortDisplayName  Team Name   Nickname   Location  \
0  Arizona Cardinals             Cardinals  Cardinals  Cardinals    Arizona   
1    Atlanta Falcons               Falcons    Falcons    Falcons    Atlanta   
2   Baltimore Ravens                Ravens     Ravens     Ravens  Baltimore   
3      Buffalo Bills                 Bills      Bills      Bills    Buffalo   
4  Carolina Panthers              Panthers   Panthers   Panthers   Carolina   

  team.color team.alternateColor  team.isActive  team.isAllStar  \
0     a40227              ffffff           True           Fal

### Part 2 Find Duplicates

In [107]:
# Check for duplicates based on the actual column names 
duplicates = df[df.duplicated(subset=['Team Name', 'Location'], keep=False)]

# Display the duplicated rows
if not duplicates.empty:
    print("Duplicate rows found:")
    print(duplicates)
else:
    print("No duplicates found.")

No duplicates found.


### Part 3 Format data into a more readable format

In [72]:
# Select important columns
df_cleaned = df_cleaned[['Team Name', 'Nickname', 'Location']]  

# Display the cleaned and formatted DataFrame
print("Formatted DataFrame:")
print(df_cleaned.head())

Formatted DataFrame:
   Team Name   Nickname   Location
0  Cardinals  Cardinals    Arizona
1    Falcons    Falcons    Atlanta
2     Ravens     Ravens  Baltimore
3      Bills      Bills    Buffalo
4   Panthers   Panthers   Carolina


### Part 4 Identify outliers and bad data

In [81]:
import numpy as np  # Import numpy to handle numerical operations
from scipy import stats  # Import scipy for Z-score calculation

# Identify outliers using Z-score
numeric_columns = ['team.id'] 
df_cleaned[numeric_columns] = df_cleaned[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Calculate Z-scores
z_scores = np.abs(stats.zscore(df_cleaned[numeric_columns]))
outliers = df_cleaned[(z_scores > 3).any(axis=1)]  # Identifies rows where Z-score > 3 (outliers)

print("Outliers based on Z-score:\n", outliers)

# identify outliers using IQR (Interquartile Range)
Q1 = df_cleaned[numeric_columns].quantile(0.25)
Q3 = df_cleaned[numeric_columns].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as data points below Q1 - 1.5*IQR or above Q3 + 1.5*IQR
outliers_iqr = df_cleaned[((df_cleaned[numeric_columns] < (Q1 - 1.5 * IQR)) | 
                           (df_cleaned[numeric_columns] > (Q3 + 1.5 * IQR))).any(axis=1)]

print("Outliers based on IQR:\n", outliers_iqr)

# Cleaning and formatting
df_final = df_cleaned.drop(outliers.index)  # Remove outliers if needed
print("Cleaned DataFrame:\n", df_final.head())

Outliers based on Z-score:
 Empty DataFrame
Columns: [team.id, team.uid, team.slug, team.abbreviation, team.displayName, team.shortDisplayName, Team Name, Nickname, Location, team.color, team.alternateColor, team.isActive, team.isAllStar, team.logos, team.links]
Index: []
Outliers based on IQR:
 Empty DataFrame
Columns: [team.id, team.uid, team.slug, team.abbreviation, team.displayName, team.shortDisplayName, Team Name, Nickname, Location, team.color, team.alternateColor, team.isActive, team.isAllStar, team.logos, team.links]
Index: []
Cleaned DataFrame:
    team.id        team.uid          team.slug team.abbreviation  \
0       22  s:20~l:28~t:22  arizona-cardinals               ARI   
1        1   s:20~l:28~t:1    atlanta-falcons               ATL   
2       33  s:20~l:28~t:33   baltimore-ravens               BAL   
3        2   s:20~l:28~t:2      buffalo-bills               BUF   
4       29  s:20~l:28~t:29  carolina-panthers               CAR   

    team.displayName team.shortDisp

### Part 5 Fix casing or inconsistent values

In [101]:
# Fix casing and remove extra whitespace in relevant columns
df['Team Name'] = df['Team Name'].str.strip().str.title()  # Title case for team names
df['Nickname'] = df['Nickname'].str.strip().str.lower()  # Lowercase for nicknames
df['Location'] = df['Location'].str.strip().str.title()  # Title case for location

# Handle inconsistent values 
df['Location'] = df['Location'].replace({
    'Sf': 'San Francisco', 
    'San Fran': 'San Francisco'
})

# Display the cleaned and standardized DataFrame
print("Cleaned and standardized DataFrame:")
print(df.head())

Cleaned and standardized DataFrame:
  team.id        team.uid          team.slug team.abbreviation  \
0      22  s:20~l:28~t:22  arizona-cardinals               ARI   
1       1   s:20~l:28~t:1    atlanta-falcons               ATL   
2      33  s:20~l:28~t:33   baltimore-ravens               BAL   
3       2   s:20~l:28~t:2      buffalo-bills               BUF   
4      29  s:20~l:28~t:29  carolina-panthers               CAR   

    team.displayName team.shortDisplayName  Team Name   Nickname   Location  \
0  Arizona Cardinals             Cardinals  Cardinals  cardinals    Arizona   
1    Atlanta Falcons               Falcons    Falcons    falcons    Atlanta   
2   Baltimore Ravens                Ravens     Ravens     ravens  Baltimore   
3      Buffalo Bills                 Bills      Bills      bills    Buffalo   
4  Carolina Panthers              Panthers   Panthers   panthers   Carolina   

  team.color team.alternateColor  team.isActive  team.isAllStar  \
0     a40227             